<center>

# $\textbf{Gross Domestic Product }$

<center>

### $\textbf{Code}$

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, lit, array, struct
import time

In [2]:
inicio = time.time()

In [3]:
spark = SparkSession.builder.appName('GDP').master("local").enableHiveSupport().getOrCreate()
spark

24/03/14 16:32:16 WARN Utils: Your hostname, Francisco-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.191.2.158 instead (on interface en0)
24/03/14 16:32:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 16:32:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("../Files/GDP.csv", format="csv", sep=",", inferschema="true", header="true")

In [5]:
# Assuming df is properly defined DataFrame
df = df.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("gdp")) 
    for year in range(2000, 2025)
])).alias("data")).selectExpr("country", "data.year", "data.gdp")

In [6]:
# Cast columns to their desired types
df = df.withColumn("country", col("country").cast("string"))
df = df.withColumn("year", col("year").cast("int"))
df = df.withColumn("gdp", col("gdp").cast("double"))

In [7]:
# Filter data for years greater than 2010 and lower than 2024
df = df.filter(df["year"] > 2010)
df = df.filter(df["year"] < 2024)

In [8]:
# Order by country and then by year
df = df.orderBy("country", "year")

### $\textbf{Pre-Processing}$

In [9]:
# Rename the column
df = df.withColumnRenamed("China, People\'s Republic of","China")
df = df.withColumnRenamed("Gambia, The","Gambia")
df = df.withColumnRenamed("Micronesia, Fed. States of","Micronesia")
df = df.withColumnRenamed("South Sudan, Republic of","South Sudan")
df = df.withColumnRenamed("Taiwan Province of China","Taiwan")
df = df.withColumnRenamed("Türkiye, Republic of","Turkiye")
df = df.withColumnRenamed("Bahamas, The","Bahamas")
df = df.withColumnRenamed("Syria","Syrian Arab Republic")
df = df.withColumnRenamed("Czech Republic","Czechia")
df = df.withColumnRenamed("Korea, Republic of","South Korea")
df = df.withColumnRenamed("Côte d\'Ivoire","Cote d\'Ivoire")
df = df.withColumnRenamed("Hong Kong SAR","Hong Kong")
df = df.withColumnRenamed("Lao P.D.R.","Lao PDR")
df = df.withColumnRenamed("Congo, Republic of ","Republic of the Congo")
df = df.withColumnRenamed("Congo, Dem. Rep. of the","Democratic Republic of the Congo")
df = df.withColumnRenamed("Pacific Islands ","Pacific island small states")
df = df.withColumnRenamed("North Macedonia ","North Macedonia")

In [10]:
# Columns to drop
countries_to_remove = ["ASEAN-5","Advanced economies", "Africa (Region)","Asia and Pacific", "Australia and New Zealand","Central America",
        "Central Asia and the Caucasus","©IMF, 2023","East Asia","Eastern Europe ","Emerging and Developing Asia",
        "Emerging and Developing Europe","Emerging market and developing economies","Euro area","Europe","European Union",
        "Latin America and the Caribbean","Macao SAR","Major advanced economies (G7)","Middle East (Region)","Middle East and Central Asia",
        "North Africa","North America","North Macedonia","Other advanced economies","South America","South Asia","Southeast Asia",
        "Sub-Saharan Africa","Sub-Saharan Africa (Region)","Western Europe","Western Hemisphere (Region)","World"]

df = df.filter(~df["country"].isin(countries_to_remove))

In [11]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("../FilesParquet/GDP.parquet" )
spark.read.parquet("../FilesParquet/GDP.parquet").show()
spark.stop()

24/03/14 16:32:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----+--------+
|    country|year|     gdp|
+-----------+----+--------+
|Afghanistan|2011|   714.7|
|Afghanistan|2012| 784.611|
|Afghanistan|2013| 754.402|
|Afghanistan|2014| 746.922|
|Afghanistan|2015| 705.597|
|Afghanistan|2016| 617.126|
|Afghanistan|2017| 635.789|
|Afghanistan|2018| 582.323|
|Afghanistan|2019| 586.204|
|Afghanistan|2020| 611.268|
|Afghanistan|2021| 443.385|
|Afghanistan|2022|     0.0|
|Afghanistan|2023|     0.0|
|    Albania|2011| 4439.89|
|    Albania|2012|4248.909|
|    Albania|2013|  4415.6|
|    Albania|2014|4584.919|
|    Albania|2015|3953.611|
|    Albania|2016|4124.405|
|    Albania|2017|4525.887|
+-----------+----+--------+
only showing top 20 rows



In [12]:
fim = time.time()
final = fim - inicio
print(final)

6.553692817687988
